Import libraries

In [69]:
import sys
import psycopg2
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

Import Database variables

In [70]:
ELEPHANTSQL_DATABASE = getenv('ELEPHANTSQL_DATABASE')
ELEPHANTSQL_USERNAME = getenv('ELEPHANTSQL_USERNAME')
ELEPHANTSQL_PASSWORD = getenv('ELEPHANTSQL_PASSWORD')
ELEPHANTSQL_HOST = getenv('ELEPHANTSQL_HOST')

SQL Connect

In [71]:
def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client

SQL Create Table

In [72]:
def create_table(elephantsql_client, command):
    '''Creating table with given input command'''    
    try:
        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Execute Command

In [73]:
def execute_command(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    try:
        # Create cursor object
        cur = elephantsql_client.cursor()

        # Execute command
        cur.execute(insert_req)

        # Commit changes
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:

        print("Error: %s" % error)

        # Reset to prev database state
        elephantsql_client.rollback()

        # Close cursor object
        cur.close()
        return 1
        
    cur.close()

Create usernames Table

In [76]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Building Inital user tweet tables
command = '''CREATE TABLE IF NOT EXISTS usernames_table (username               varchar(25),
                                                         questions_right        varchar(50),
                                                         questions_wrong        varchar(50),
                                                         question_accuracy      varchar(10),
                                                         total_time             varchar(50),
                                                         PRIMARY KEY(username))'''
create_table(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


Get a Value

In [7]:
def get_value(elephantsql_client, command):
    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    try:
        # Execute commands in order
        cur.execute(command)

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 'failed'

    # Get cur value
    returned = cur.fetchall()
    
    # Close communication with the PostgreSQL database server
    cur.close()

    return returned

Drop Table

In [67]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

command = '''DROP TABLE usernames_table;'''

execute_command(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


Database filling

In [37]:
# Json response data
username = 'tyler'
questions_right = 2
questions_wrong = 3
question_accuracy = 40
total_time = 25


# Testing username/ grabbing elements
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

command = '''SELECT * FROM usernames_table WHERE username = '{}';'''.format(username)
username_package = get_value(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

# Quary failed
if username_package == []:
    # Username has not yet been created

    # Database connection
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    command = "INSERT INTO usernames_table (username, questions_right, questions_wrong, question_accuracy, total_time) VALUES ('{}', '{}', '{}', '{}', '{}')".format(username,
                                                                                                                                                                     questions_right,
                                                                                                                                                                     questions_wrong,
                                                                                                                                                                     question_accuracy,
                                                                                                                                                                     total_time)
    execute_command(elephantsql_client, command)

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')

#Quarry succeded
else:
    questions_right += int(username_package[0][1])
    questions_wrong += int(username_package[0][2])
    question_accuracy = (questions_right / (questions_right + questions_wrong)) * 100
    total_time += int(username_package[0][4])

    # Convert to string
    questions_right = str(questions_right)
    questions_wrong = str(questions_wrong)
    question_accuracy = str(question_accuracy)
    total_time = str(total_time)

    # Database connection
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    command = "UPDATE usernames_table SET questions_right = '{}', questions_wrong = '{}', question_accuracy = '{}', total_time = '{}' WHERE username = '{}';".format(questions_right,
                                                                                                                                                                      questions_wrong,
                                                                                                                                                                      question_accuracy,
                                                                                                                                                                      total_time,
                                                                                                                                                                      username)
    execute_command(elephantsql_client, command)

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')





Connecting to the PostgreSQL database...
Connection is closed.
Connecting to the PostgreSQL database...
Connection is closed.


Return List of a table

In [ ]:
def get_value(elephantsql_client, command):
    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    try:
        # Execute commands in order
        cur.execute(command)

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1

    # Get cur value
    returned = cur.fetchall()
    
    # Close communication with the PostgreSQL database server
    cur.close()

    return returned

Getting Sorted Table

In [66]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

command = '''SELECT * FROM usernames_table ORDER BY question_accuracy DESC'''
usernames_package = get_value(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


In [65]:
usernames_package

[('superman', '6', '9', '40.0', '60'),
 ('shaem', '1', '4', '20', '15'),
 ('kid', '15', '0', '100.0', '66'),
 ('human', '5', '0', '100', '22'),
 ('dave', '5', '0', '100', '22')]